# Building a Digital Orrery
## An exercise in Object Oriented Programming

**Version 0.1**

It is your goal in this exercise to construct a Digital Orrery. An [orrery](https://en.wikipedia.org/wiki/Orrery) is a mechanical model of the Solar System. Here, we will generalize this to anything that is mechanically similar to *the* solar system: a collection of things bound gravitationally. 


<img src="https://upload.wikimedia.org/wikipedia/commons/4/48/Grand_orrery_in_Putnam_Gallery%2C_2009-11-24.jpg" alt="Orrery" width="600"/>
(image: wikimedia)


* * *

By J. S. Oishi (Bates College)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from matplotlib import animation, rc
from IPython.display import HTML

%matplotlib notebook

## Problem 1) Building a basic set of objects

Our first task is to map our problem onto a set of **objects** that we **instantiate** (that is, make **instances** of) in order to solve our problem.

Let's outline the scope of our problem.

A solar system exists in a Universe; here we can ignore the gravitational perturbation on the Solar System from the rest of the Universe. Our model will consist of a small number of bodies containing mass. It might also contain bodies without mass, so called "test particles".

The problem to be solved numerically is the gravitational N-body problem,

$$\ddot{\mathbf{r}}_i = -G\sum_{i \ne j} \frac{m_j \mathbf{r}_{ij}}{r_{ij}^3},$$

where $\mathbf{r}_{ij} \equiv \mathbf{r_i} - \mathbf{r_j}$. This task itself can be broken into two components: 

* the force calculation
* the ODE integrator to advance $\mathbf{r}_i$ and $\dot{\mathbf{r}}_i$ forward in time

**Problem 1a**

In disucssion with a classmate, sketch out a set of classes that you will need to complete this project. Don't worry about things like numerical integrators yet. 

Also, sketch out interfaces (start with the constructor), but don't worry about writing code right now.

*Once you're done, find me and I'll give you the minimal list of objects.*

In [ ]:
class particle():
    def __init__(self, position, mass, velocity):
        self.position = position
        self.mass = mass
        self.velocity = velocity
        
    def calc_net_force(self, list_of_particles):
        #net_force = np.sum(np.array([self.calc_force(p) for p in list_of_particles]))
        net_force = np.array([0., 0.])
        for p in list_of_particles:
            force = self.calc_force(p)
            #print(force)
            net_force += force     
        #print(" ")
        return net_force
    
    def calc_force(self, other_particle):
        G = 6.67e-11 #SI units
        dist = self.calc_distance(other_particle)
        if dist == 0.0: dist = 1.e-3
        force = G * self.mass * other_particle.mass / dist **3 * (other_particle.position - self.position)
        
        return force
        
    def calc_distance(self, other_particle):
        dist = np.sqrt((self.position[0] - other_particle.position[0])**2 + \
                       (self.position[1] - other_particle.position[1])**2)
        return dist

    def update_velocity_and_position(self, list_of_other_particles, delta_t):
        net_force = self.calc_net_force(list_of_other_particles)
        acc = net_force / self.mass
        #new velocity
        new_vel = acc * delta_t + self.velocity
        self.velocity = new_vel
        #new position
        new_pos = 0.5 * acc * delta_t**2 + self.velocity * delta_t + self.position
        self.position = new_pos
        return
        
    

In [ ]:
p1 = particle(np.array([0.,0.]), 1.989e30, np.array([0.,0.])) #sun
p2 = particle(np.array([0.,1.496e11]), 5.972e24, np.array([3.0e4,0.])) #earth
p3 = particle(np.array([0.,-7.785e11]), 1.898e27, np.array([-4.705e4,0.])) #jupiter

In [ ]:
p1.calc_net_force([p2,p3])

In [ ]:
class Universe():
    def __init__(self, particles, time_step, total_time):
        self.particles = particles
        self.time_step = time_step
        self.total_time = total_time
        self.current_time = 0.0
        self.x_list = []
        self.y_list = []
                
    def advance(self):
        for p in self.particles:
            p.update_velocity_and_position(self.particles, self.time_step)
        self.current_time += time_step
        #return current positons for plotting
        x = [p.position[0] for p in self.particles]
        y = [p.position[1] for p in self.particles]
        return x, y
    
    def run(self):
        while self.current_time < self.total_time:
            ## do the animation in here
            x, y = self.advance()
            
            #track positions
            self.x_list.append(x)
            self.y_list.append(y)
            """ 
            anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)
            """
            
            sys.stdout.write('\rCurrent Time: %.5f' %self.current_time)
            sys.stdout.flush()
            

In [ ]:
particles = [p1, p2, p3]
time_step = 1.
total_time = 100.

U = Universe(particles, time_step, total_time)
U.run()

In [ ]:
import pandas as pd

In [ ]:
x_df = pd.DataFrame(data=U.x_list, columns=[str(k+1) for k in range(len(particles))])
y_df = pd.DataFrame(data=U.y_list, columns=[str(k+1) for k in range(len(particles))])

In [ ]:
plt.figure()

for ii in range(len(particles)):
    name = str(ii+1)
    plt.scatter(x_df[name], y_df[name], label=name)

plt.show()

**Problem 1b**

Wire them up! Now that you have the list, try them out. Python makes use of duck typing, you should too. That is, if your object has a mass `m`, a position `r` and a velocity `rdot`, it *is* a Body.

In [ ]:
r0 = np.array([0,0,0])
rdot0 = np.array([0,0,0])

In [ ]:
b = Body(1,r0, rdot0)

## Problem 2

Now, we code the numerical algorithms. We're going to do the most simple things possible: a *brute force* ("direct N-Body" if you're feeling fancy) force calculation, and a leapfrog time integrator.

The leapfrog scheme is an explicit, second order scheme given by

$$r_{i+1} = r_{i} + v_{i} \Delta t + \frac{\Delta t^2}{2} a_{i}$$

$$v_{i+1} = v_{i} + \frac{\Delta t}{2} (a_{i} + a_{i+1}),$$

where $\Delta t$ is the time step (which we'll just keep constant), and the subscript refers to the *iteration* number $i$. 

Note that this scheme requires a force update *in between* calculating $r_{i+1}$ and $v_{i+1}$.

**Problem 2a** 

Write a method that implements the force integrator. Test it on simple cases:
 * two equal 1 $M_\odot$ objects in your universe, 1 AU apart
 * a $1\ M_\odot$ object and a $1\ M_{\oplus}$ object, 1 AU apart

**Problem 2b**
Write the leapfrog integration as a method in the `Universe` class. Test it on one particle with no force (what should it do?)

**Problem 2c** 
 
Wire it all up! Try a 3-body calculation of the Earth-Sun-Moon system. Try the Earth-Jupiter-Sun system! 

## Challenge Problem

* Construct a visualization method for the `Universe` class
* Read about the Fast Multipole Method (FMM) [here](https://math.nyu.edu/faculty/greengar/shortcourse_fmm.pdf) and implement one for the force calculation

In [ ]:
# good luck!